In [ ]:
from importnb import Notebook
with Notebook():
    import news_analysis
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import pickle
from glob import glob
import re
from copy import deepcopy

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import hyperopt
from hyperopt import hp

In [ ]:
PATH_TO_DATA = 'data/news_models/'

In [ ]:
df_news_paths = glob(PATH_TO_DATA+'dfnews*.pkl')
df_votes_paths = glob(PATH_TO_DATA+'dfvotes*.pkl')

df_news_dates = np.array([datetime.datetime.strptime(
                                re.findall(pattern='([0-9]{8}T[0-9]{4})', string=path)[0], '%Y%m%dT%H%M') 
                         for path in df_news_paths])
df_votes_dates = np.array([datetime.datetime.strptime(
                                re.findall(pattern='([0-9]{8}T[0-9]{4})', string=path)[0], '%Y%m%dT%H%M') 
                         for path in df_votes_paths])

last_df_news = PATH_TO_DATA+'dfnews'+df_news_dates.max().strftime("%Y%m%dT%H%M")+'.pkl'
last_df_votes = PATH_TO_DATA+'dfvotes'+df_votes_dates.max().strftime("%Y%m%dT%H%M")+'.pkl'

df_news, df_votes = news_analysis.load_news_and_votes(file_news=last_df_news,file_votes=last_df_votes)
df_news.sort_index(by='created_at', inplace=True)

In [ ]:
df_prices = pd.read_csv(PATH_TO_DATA+'prices.tmp')
df_prices.head()

In [ ]:
titles_emb = news_analysis.get_doc2vec_embeddings(df_news.title.values, vector_size=10, window=2, min_count=1, workers=4)

In [ ]:
def make_X_Y(titles_embeddings, prices0, prices1, window_width=10, window_stride=10):
    X = []
    Y = []
    max_price = max(prices0.max(), prices1.max())
    for i in range(window_width, titles_embeddings.shape[0], window_stride):
        X.append(titles_embeddings[i-window_width:i].flatten())
        Y.append(prices1[i] - prices0[i])
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [ ]:
def score(y_pred, y_test):
    score_up = 0
    score_down = 0
    for i in range(len(y_pred)):
        if y_pred[i] > 0:
            score_up += float(y_test[i])
        elif y_pred[i] < 0:
            score_down -= float(y_test[i])
    return score_up, score_down

def score_mean(y_pred, y_test, score_tuple=None):
    score_up, score_down = score(y_pred, y_test) if type(score_tuple) == type(None) else score_tuple
    score_up /= len(y_pred)
    score_down /= len(y_pred)
    return score_up, score_down

def score_f1(y_pred, y_test, score_tuple=None):
    score_up, score_down = score(y_pred, y_test) if type(score_tuple) == type(None) else score_tuple
    score_up /= len(y_pred)
    score_down /= len(y_pred)
    if score_up > 0 and score_down > 0:
        f1 = 2 * score_up * score_down / (score_up + score_down)
    else:
        f1 = min(score_up, score_down)
    return f1

In [ ]:
def cv_score(model, X, y, scoring_func, k_folds=10, verbose=0):
    print('CV splitting') if verbose > 0 else None 
    X = np.array(X)
    y = np.array(y)
    if len(y.shape) == 1:
        y = y.copy().reshape(-1,1)
    
    data = np.concatenate((X, y), axis=1)
    np.random.shuffle(data)
    X, y = data[:,:X.shape[1]], data[:,X.shape[1]:]
    
    test_sizes = np.ones(k_folds) * (X.shape[0] // k_folds) + \
                 np.concatenate([np.ones(X.shape[0] % k_folds), np.zeros(k_folds - X.shape[0] % k_folds)])
    
    print('Starting CV') if verbose > 0 else None 
    scores = []
    for i in range(k_folds):
        test_indexes = np.arange(test_sizes[:i].sum(), test_sizes[:i+1].sum()).astype(int)
        train_indexes = np.concatenate([np.arange(test_indexes[0]), np.arange(test_indexes[-1]+1,X.shape[0])]).astype(int)
        
        fitted_model = deepcopy(model)
        fitted_model.fit(X[train_indexes], y[train_indexes])
        y_pred = fitted_model.predict(X[test_indexes])
        if type(scoring_func) == list:
            scores.append([score(y_pred, y[test_indexes]) for score in scoring_func])
        else:
            scores.append(scoring_func(y_pred, y[test_indexes]))
        if verbose > 0:
            print('Score:',scores[-1])
    
    return scores

In [ ]:
period = 5 * 60
lag = 6 * 12 * period
dates0 = news_analysis.to_datetime(df_news.created_at.values)
dates1 = dates0 + datetime.timedelta(0, lag)
prices0 = news_analysis.get_prices_at_date(dates0, df_prices)
prices1 = news_analysis.get_prices_at_date(dates1, df_prices)

In [ ]:
if np.any(prices1 == 0):
    end_ix = (prices1 == 0).argmax()
else:
    end_ix = prices1.shape[0]
    
titles_emb = titles_emb[:end_ix]
prices0 = prices0[:end_ix]
prices1 = prices1[:end_ix]
dates0 = dates0[:end_ix]
dates1 = dates1[:end_ix]
X, Y = make_X_Y(titles_emb, prices0, prices1, window_stride=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)

In [ ]:
def integrate(diff):
    x0 = 0
    x = [x0]
    for d in diff:
        x.append(x[-1] + d)
    return np.array(x)

In [ ]:
class ObjectiveFunction:

    def __init__(self, X, y, X_test, y_test):
        self.X = X
        self.y = y
        self.X_test = X_test
        self.y_test = y_test

    # Calculate cross validation score (default is10-fold CV)
    def __call__(self, X, cv=10):
        # Make the SVM classifire with  the specific 'C' and 'degree'
        xgb = XGBRegressor(max_depth=int(X['max_depth']),
                           #learning_rate=X['learning_rate'],
                           n_estimators=int(10**X['n_estimators']),
                           n_jobs=-1)
        
        xgb.fit(self.X, self.y)
        y_pred = xgb.predict(self.X_test)
        s = score(y_pred, self.y_test)
        s_mean = score_mean(y_pred, self.y_test, s)
        f1 = score_f1(y_pred, self.y_test, s)
        print(X)
        print('Sum score:', sum(s))
        print('Sum score up and down:', s)
        print('Mean score up and down:', score_mean)
        print('F1:', f1)
        return - f1    # for minimization

In [ ]:
# Definition of design variables
parameter_space = {#'learning_rate':hp.loguniform("learning_rate", 0.01, 0.5),
                   'max_depth': hp.quniform('max_depth', 1, 5, q=1),
                   'n_estimators': hp.uniform('n_estimators', 1, 3)}

# Objective function
f = ObjectiveFunction(X_train, Y_train, X_test, Y_test)
trials = hyperopt.Trials()

best = hyperopt.fmin(f, parameter_space, 
                     algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, verbose=1)
print("best estimate parameters", best)

In [ ]:
%%time
xgb = XGBRegressor(max_depth=int(2.0),
                           #learning_rate=X['learning_rate'],
                           n_estimators=int(10**1.690026595292681),
                           n_jobs=-1)
scores = cv_score(xgb, X_train, Y_train, [score, score_mean, score_f1], k_folds=10)

In [ ]:

scores

In [ ]:
xgb.fit(X_train, Y_train)
Y_pred = xgb.predict(X_test)

In [ ]:
score(Y_pred, Y_test)

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(dates1[-Y_pred.shape[0]:], prices0[-Y_pred.shape[0]:] + Y_pred)

# plt.plot(dates1, prices1)
plt.plot(dates1[-Y_pred.shape[0]:], prices1[-Y_pred.shape[0]:])
plt.legend(['prediction', 'true'])
plt.grid()

In [ ]:
gain = []
thresholds = np.arange(30)
for threshold in thresholds:
    gain.append(integrate(Y_test[np.abs(Y_pred) > threshold] * np.sign(Y_pred[np.abs(Y_pred) > threshold]) \
                            - 0.001 * (prices0[-Y_test.shape[0]:][np.abs(Y_pred) > threshold]))[-1])

gain

In [ ]:
plt.figure(figsize=(14,10))
threshold = 10
# plt.plot(integrate(Y_test[np.abs(Y_pred) > threshold] * np.sign(Y_pred[np.abs(Y_pred) > threshold]) \
#                    - 0.001 * (prices0[-Y_test.shape[0]:][np.abs(Y_pred) > threshold] + prices1[-Y_test.shape[0]:][np.abs(Y_pred) > threshold])))
plt.plot(integrate(Y_test[np.abs(Y_pred) > threshold] * np.sign(Y_pred[np.abs(Y_pred) > threshold]) \
                   - 0.001 * (prices0[-Y_test.shape[0]:][np.abs(Y_pred) > threshold])))
plt.legend(['prediction', 'true'])

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(dates1[-Y_pred.shape[0]:], integrate(Y_pred))
plt.plot(dates1[-Y_test.shape[0]:], integrate(Y_test))
plt.legend(['prediction', 'true'])

In [ ]:
plt.hist((Y - Y.mean()) / Y.std(), bins=np.linspace(-2,2, 40))

In [ ]:
def make_dataset(df_news, df_prices, 
                 lag=6*60*60, 
                 vector_size=10, window=2, min_count=1, 
                 window_width=10, window_stride=1, 
                 workers=4, verbose=0):
    
    print('Extracting dates') if verbose>0 else None
    dates0 = news_analysis.to_datetime(df_news.created_at.values)
    dates1 = dates0 + datetime.timedelta(0, lag)
    print('Extracting prices') if verbose>0 else None
    prices0 = news_analysis.get_prices_at_date(dates0, df_prices)
    prices1 = news_analysis.get_prices_at_date(dates1, df_prices)
    
    if np.any(prices1 == 0):
        end_ix = (prices1 == 0).argmax()
    else:
        end_ix = prices1.shape[0]
        
    print('Creating titles embeddings') if verbose>0 else None
    titles_emb = news_analysis.get_doc2vec_embeddings(df_news.title.values, 
                                                      vector_size=vector_size, 
                                                      window=window, 
                                                      min_count=min_count, 
                                                      workers=workers)
    print('Creating dataset') if verbose>0 else None
    titles_emb = titles_emb[:end_ix]
    prices0 = prices0[:end_ix]
    prices1 = prices1[:end_ix]
    dates0 = dates0[:end_ix]
    dates1 = dates1[:end_ix]
    X, Y = make_X_Y(titles_emb, prices0, prices1, window_width=window_width, window_stride=window_stride)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)
    
    return dates0, dates1, prices0, prices1, X_train, X_test, Y_train, Y_test

In [ ]:
class ObjectiveFunction_XGB:

    def __init__(self, df_news, df_prices):
        self.df_news = df_news
        self.df_prices = df_prices

    # Calculate cross validation score (default is10-fold CV)
    def __call__(self, X, cv=10):
        dates0, dates1, prices0, prices1, X_train, X_test, Y_train, Y_test = make_dataset(
                                                        self.df_news,
                                                        self.df_prices,
                                                        lag=int(X['lag'])*12*5*60,
                                                        vector_size=10,#int(X['vector_size']),
                                                        window=2,#int(X['window']),
                                                        min_count=int(X['min_count']),
                                                        window_width=7,#int(X['window_width']),
                                                        window_stride=1,#int(X['window_stride']),
                                                        workers=4,
                                                        verbose=0)
        
        xgb = XGBRegressor(max_depth=int(X['max_depth']),
                           #learning_rate=X['learning_rate'],
                           n_estimators=int(10**X['n_estimators']),
                           n_jobs=4)
        
        scores = cv_score(xgb, X_train, Y_train, (lambda x, y: score_mean(x, y)[1]), k_folds=10, verbose=1)
        print(X)
        mean_score = np.mean(scores)
        print('Mean score:', mean_score)
        print()
        return - mean_score    # for minimization

In [ ]:
%%time 
# Definition of design variables
parameter_space = {#'learning_rate':hp.loguniform("learning_rate", 0.01, 0.5),
                    'max_depth': hp.quniform('max_depth', 2, 5, q=1),
                    'n_estimators': hp.uniform('n_estimators', 1, 1.2), 
                    'lag': hp.quniform('lag', 4, 12, q=2),
#                     'vector_size':hp.quniform('vector_size', 6, 10, q=1),
#                     'window':hp.quniform('window', 2,5,q=1),
                    'min_count':hp.quniform('min_count', 1,3,q=1)}
#                     'window_width':hp.quniform('window_width', 6,10,q=1),
#                     'window_stride':hp.quniform('window_stride', 1,5,q=1)}

# Objective function
f = ObjectiveFunction_XGB(df_news, df_prices)
trials = hyperopt.Trials()

best = hyperopt.fmin(f, parameter_space, 
                     algo=hyperopt.tpe.suggest, max_evals=25, trials=trials, verbose=1)
print("best estimate parameters", best)

In [ ]:
dates0, dates1, prices0, prices1, X_train, X_test, Y_train, Y_test = make_dataset(df_news, df_prices,
                                                                                  lag=8*12*5*60,
                                                                                   vector_size=10,
                                                                                  window=2,
                                                                                  min_count=3,
                                                                                  window_width=7,
                                                                                  window_stride=1)

In [ ]:
%%time
xgb = XGBRegressor(n_jobs=-1, max_depth=2, n_estimators=int(10**1.1982429076195422))
xgb.fit(X_train, Y_train)

In [ ]:
Y_pred = xgb.predict(X_test)

In [ ]:
score(Y_pred, Y_test)

In [ ]:
score_mean(Y_pred, Y_test)

In [ ]:
score_f1(Y_pred, Y_test)

In [ ]:
gain = []
thresholds = np.arange(10)
for threshold in thresholds:
    gain.append(integrate(Y_test[-Y_pred > threshold] * np.sign(Y_pred[-Y_pred > threshold]) \
                   - 0.001 * (prices0[-Y_test.shape[0]:][-Y_pred > threshold]))[-1])
np.array(gain).argmax()

In [ ]:
plt.figure(figsize=(14,10))
threshold = 4
# plt.plot(integrate(Y_test[np.abs(Y_pred) > threshold] * np.sign(Y_pred[np.abs(Y_pred) > threshold]) \
#                    - 0.001 * (prices0[-Y_test.shape[0]:][np.abs(Y_pred) > threshold] + prices1[-Y_test.shape[0]:][np.abs(Y_pred) > threshold])))
# plt.plot(integrate(Y_test[np.abs(Y_pred) > threshold] * np.sign(Y_pred[np.abs(Y_pred) > threshold]) \
#                    - 0.001 * (prices0[-Y_test.shape[0]:][np.abs(Y_pred) > threshold])))
integral = integrate(Y_test[-Y_pred > threshold] * np.sign(Y_pred[-Y_pred > threshold]) \
                   - 0.001 * (prices0[-Y_test.shape[0]:][-Y_pred > threshold]))
plt.plot(integral)
plt.grid()
print(integral[-1])

In [ ]:
df = pd.read_csv(PATH_TO_DATA+'logs/XGB_20181028T1603.csv')
df.sort_values(by='score', ascending=False)

In [ ]:
df.iloc[:,1:].corr()

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [ ]:
m1 = smf.ols('score ~ lag + max_depth + min_count + n_estimators + vector_size + window + window_stride + window_width', 
             data=df)
fitted = m1.fit()
fitted.summary()